#### regression

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CA_Housing_builtin").getOrCreate()

import pandas as pd
from sklearn.datasets import fetch_california_housing
cal = fetch_california_housing()   
pdf = pd.DataFrame(cal.data, columns=cal.feature_names)
pdf['medianHouseValue'] = cal.target    

housing = spark.createDataFrame(pdf)

housing = housing.na.drop(how="any")

from pyspark.ml.feature import RFormula
rf = RFormula(formula="medianHouseValue ~ .")

from pyspark.ml.regression import LinearRegression
lr = LinearRegression()

from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[rf, lr])

from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="rmse") 

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
params = ParamGridBuilder().addGrid(lr.regParam, [0, 0.1, 0.2]).build()
cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=params,
                    evaluator=evaluator,
                    numFolds=4)

train, test = housing.randomSplit([0.7, 0.3])
model = cv.fit(train)

print("训练集 RMSE:", evaluator.evaluate(model.transform(train)))
print("测试集 RMSE:", evaluator.evaluate(model.transform(test)))

25/10/07 21:16:24 WARN Instrumentation: [2071894b] regParam is zero, which might cause numerical instability and overfitting.
25/10/07 21:16:24 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/10/07 21:16:24 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
25/10/07 21:16:26 WARN Instrumentation: [2fd36b53] regParam is zero, which might cause numerical instability and overfitting.
25/10/07 21:16:27 WARN Instrumentation: [80610d62] regParam is zero, which might cause numerical instability and overfitting.
25/10/07 21:16:28 WARN Instrumentation: [f72c9d9a] regParam is zero, which might cause numerical instability and overfitting.
25/10/07 21:16:29 WARN Instrumentation: [20564c99] regParam is zero, which might cause numerical instability and overfitting.


训练集 RMSE: 0.7249646889209516
测试集 RMSE: 0.7232588285989255
